In [1]:
!git clone https://huggingface.co/microsoft/trocr-large-handwritten

Cloning into 'trocr-large-handwritten'...
remote: Enumerating objects: 33, done.
remote: Total 33 (delta 0), reused 0 (delta 0), pack-reused 33
Unpacking objects: 100% (33/33), 540.03 KiB | 1.88 MiB/s, done.


In [2]:
#wget csak linuxon műkszik
!wget "https://huggingface.co/datasets/zhixiaoni/CROHME_selected_Train_2014_png/resolve/refs%2Fconvert%2Fparquet/default/train/0000.parquet"

--2024-04-17 10:06:59--  https://huggingface.co/datasets/zhixiaoni/CROHME_selected_Train_2014_png/resolve/refs%2Fconvert%2Fparquet/default/train/0000.parquet
Resolving huggingface.co (huggingface.co)... 18.239.50.49, 18.239.50.16, 18.239.50.80, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.49|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/15/ba/15ba10db322e712c99d4a98d2c2e2d087ceb2ede4198f990212f3c843ff1cb0b/7d0de11d803753180c871c6ca19ecbf61f6f1ec9dd5a234f8bd089c2ee471451?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%270000.parquet%3B+filename%3D%220000.parquet%22%3B&Expires=1713607619&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMzYwNzYxOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8xNS9iYS8xNWJhMTBkYjMyMmU3MTJjOTlkNGE5OGQyYzJlMmQwODdjZWIyZWRlNDE5OGY5OTAyMTJmM2M4NDNmZjFjYjBiLzdkMGRlMTFkODAzNzUzMTgwYzg3MWM2Y2ExOWVjYmY

import cv2
import matplotlib.pyplot as plt
i

In [13]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import cv2
import numpy as np
import matplotlib.pyplot as plt
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-large-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-large-handwritten')
i_list = [0]

def OCR(row,i_list,processor,model):
    i_list[0]= i_list[0]+1
    print(i_list[0])
    nparr = np.fromstring(row["image"]["bytes"], np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    #plt.figure(figsize=(10,10)); plt.axis('off'); plt.imshow(img)
    #plt.show()
    pixel_values = processor(images=img, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(generated_text)
    return generated_text

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-large-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
import pandas as pd

df = pd.read_parquet("./0000.parquet")
df

,image
0,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...
1,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...
2,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...
3,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...
4,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...
...,...
5613,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...
5614,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...
5615,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...
5616,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...


In [ ]:
df = df.head(1000)
df["text"] = df.apply(lambda x: OCR(x, i_list,processor,model), axis=1)

1


<ipython-input-13-cbf4a8af245b>:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  nparr = np.fromstring(row["image"]["bytes"], np.uint8)
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


s - ( E.O. ( (m-2 ) " ) #
2


In [ ]:
df.shape[0]

100

In [ ]:
df.to_csv('ocr_results.csv')

In [ ]:
def get_patterns(ocr_results=['alma', 'korte', 'balma', 'almkortelma'], pattern_len=3):
    patterns = []
    for res in ocr_results:
        word_len = len(res)
        start_index = 0
        print(f"checking patterns with: {res}")
        for i in range(0, word_len-1):
            end_index = i+pattern_len
            possible_pattern = res[start_index : end_index]
            if (len(possible_pattern) != pattern_len):
                break
            print(f"\tpossible_patter: {possible_pattern}, indexes: {start_index}:{end_index}")

            #compare with other results
            for other_result in ocr_results:
                if (possible_pattern in other_result and other_result != res):
                    patterns.append(possible_pattern)
            start_index += 1
        print()
    print(f"results: {patterns}")
    return patterns

def filter_patterns(patterns, min_occurance=3):
    pattern_ocs = []
    values = {}

    for pattern in patterns:
        counter = -1
        for other in patterns:
            if (other == pattern):
                counter += 1
        if (counter >= min_occurance):
            values = {}
            values["pattern"] = pattern
            values["counter"] = counter
            pattern_ocs.append(values)
    return pattern_ocs

In [ ]:
possible_patterns = get_patterns(ocr_results=df['text'].to_list(), pattern_len=3)

In [ ]:
patterns = filter_patterns(patterns=possible_patterns, min_occurance=5)
#print(patterns)
pattern_df = pd.DataFrame.from_records(patterns)
pattern_df = pattern_df.drop_duplicates()
pattern_df.to_csv("patterns.csv", index=False)

pattern_df

,pattern,counter
0,e #,41
6,#,1701
43,# A,5
45,ear,5
47,(,1547
...,...,...
3619,te,5
3859,- O,5
3861,.S.,23
3909,# D,5
